# A Non-Parametric Bayesian Method for Inferring Hidden Causes
<a href="http://cocosci.berkeley.edu/tom/papers/ibpuai.pdf">F., Griffiths, T.L., Ghahramani, Z., 2006.<br />
Presented at the Proceedings of the Conference on Uncertainty in Artificial Intelligence.</a>

## Import

In [ ]:
import BayesianHiddenCause as bc
import imp, numpy, matplotlib.pyplot as plt
np = numpy
%matplotlib inline

In [ ]:
imp.reload(bc)

## Initialize

### Model

In [ ]:
bba = bc.BernoulliBetaAssumption(0.1, 3)

### Data

In [ ]:
X, Y, Z = bba.generate(3, 6, 200, True)
fig = bc.plot_matrix_product('i (observabes)',Z,'Z','t (trials)',Y,'Y','k (causes)',X,'X')

### Learner

In [ ]:
bba.observe(X)

In [ ]:
bba.Gibbs_prepare(10, retries=np.inf)

## Show parameters

In [ ]:
bba.Z.shape

In [ ]:
bba.N, bba.K, bba.T

In [ ]:
numpy.sum(bba.Y)

In [ ]:
bba.get_p()

In [ ]:
bba.get_theta()

## Show converged result and original data

In [ ]:
fig = bc.plot_matrix_product('i (observabes)',bba.Z,'Z','t (trials)',bba.Y,'Y','k (causes)',bba.px,'X estimated')
fig.suptitle('Estimate')
fig = bc.plot_matrix_product('i (observabes)',Z,'Z','t (trials)',Y,'Y','k (causes)',X,'X observed')
fig.suptitle('Original')

## iterate

In [ ]:
for i in range(0,100):
    bba.Gibbs_iterate()

## integrate over all models

In [ ]:
cf = X.T
p = 0.1
decay_time = np.inf
cf.shape

In [ ]:
# Train learner with first ntrial trials (equivalent weights) and see response
nfeat, ntrials = cf.T.shape
# maximal nmber of latent causes
ncaus = 3
#for ntrials in range(1,len(cf)):
learner = bc.BernoulliBetaAssumption(p, 3, decay_time=decay_time)
learner.observe(cf.astype(int).T[:,:ntrials])
tY = np.zeros((ncaus, ntrials))
cumprob = 0
infer = []
for tZ in bc.lof_iterator((nfeat, ncaus)):
    learner.Gibbs_load(tY, tZ)
    pz = np.exp(learner.logP_Z_X(tZ))
    cumprob += pz
    infer.append((pz, tZ))
# Display top 5 matches:
inferred = sorted(infer, key=lambda x: -x[0])
inferred[:5]

In [ ]:
bc.lof(Z)

### Now, go back to visualization

## Tests

In [ ]:
numpy.sum(X), numpy.sum(Y), numpy.sum(Z)

In [ ]:
bc.p0_per_sum_p(logp=[[np.log(0.25), np.log(0.25)], [np.log(0), np.log(0.75)]])

In [ ]:
bc.normalize_p(logp=[[np.log(0.25), np.log(0.25)], [np.log(1e-200), np.log(0.75)]])

In [ ]:
type('Hello')

In [ ]:
import scipy.stats
bernoulli = scipy.stats.bernoulli
b = bernoulli.rvs(p=0.2, size=(20,40))
plt.matshow(b)
plt.matshow(bc.lof(b))

In [ ]:
imp.reload(bc)

In [ ]:
for b in bc.lof_iterator(size=(2,3)):
    print(b)
    #plt.matshow(b)

In [ ]:
tmp = bc.BernoulliBetaAssumption(0.3, 3)
for i in range(0,10):
    plt.matshow(tmp.IBP(N=11))